# Perspective Correction
## Theory
Perspective correction asks the question what an image would look like, from a different camera angle. This is a basic idea in computer vision, and used extensively in video games, CG animation, special effects etc. This lab goes through the theory behind it, using an example. Work on it as a group, if possible.

## Projective Transformations
Learn more about the theory from  [Graphics Mill](https://www.graphicsmill.com/docs/gm/affine-and-projective-transformations.htm) or from [Mathematica StackExchange](https://mathematica.stackexchange.com/questions/121822/application-of-the-projection-matrix-removing-perspective-distortion)

Here are the basic transformations:  
![Transformations](https://i.stack.imgur.com/lkTbO.png "Transformations")

Looking at just two of the important ones:  
![Affine and Projective](https://www.graphicsmill.com/docs/gm/TransformationsDifference.png "Affine and Projective")

The projective transformation is not linear because it does not preserve the parallelness. Remember: In linear transformation, the unit square gets transformed to a parallelogram, which means the parallel (opposite) sides stay parallel. We will have to use the concept of [homogenous coordinates](https://en.wikipedia.org/wiki/Homogeneous_coordinates), which allows us to use matrix multiplications for some non-linear transformations as well. 

When we try to correct the perspective, we need four points:  

$$ \boldsymbol{x_i} = \{(x_0, y_0),\,\, (x_1, y_1),\,\, (x_2, y_2),\,\, (x_3, y_3)\}$$

Let's say they get projected to $ \boldsymbol{b_i}$. Since the transformation is not linear, we cannot write 

$$\boldsymbol{b_i} = \boldsymbol{A}  \boldsymbol{x_i}$$

On the other hand, using homogenous coordinates (and reusing the same symbols), we write:

$$ \boldsymbol{x_i} = \{(x_0, y_0, 1),\,\, (x_1, y_1, 1),\,\, (x_2, y_2, 1),\,\, (x_3, y_3, 1)\}$$

$ \boldsymbol{b_i}$s also have three components, the third one being 1. 

Now we can write,

$$\boldsymbol{b_i} = \boldsymbol{A}  \boldsymbol{x_i}$$
where 

$$ \boldsymbol A = \left[\begin{array}{rrr}
a_{00} & a_{01} & b_{02} \\
a_{10} & a_{11} & b_{12} \\
c_{20} & c_{21} & 1
\end{array}\right] $$

The $2\times 2$ submatrix $a_{ij}$ is a rotation. The last column ($b_{02}$ and $b_{12}$) represents a translation, and the last row ($c_{20}$ and  $c_{21}$) a projection.  

In [ ]:
vars = var('a00, a01, b02, a10, a11, b12, c20, c21')
A = matrix(SR, 3, 3, [[a00, a01, b02], [a10, a11, b12], [c20, c21, 1]])
show(A)

### Rotation
Let's take a specific example: a $\angle{45}^\circ$ rotation. This applies to the original corners of a square or rectangular shape in the image.

In [ ]:
A1 = matrix(SR, 3, 3, [[1/sqrt(2), -1/sqrt(2), 0], [1/sqrt(2), 1/sqrt(2), 0], [0, 0, 1]])
show(A1)

In [ ]:
# The four corners of a square
x0 = vector(QQ, [1, 1, 1])
x1 = vector(QQ, [5, 1, 1])
x2 = vector(QQ, [5, 5, 1])
x3 = vector(QQ, [1, 5, 1])
xList = [x0, x1, x2, x3]
show(xList)

In [ ]:
# Define a function to make the plotting easy.
# Scale by the third coordinate: Property of homogeneous coordinates
def plotList(v3List, color):
    v2List = [vector([v3[0]/v3[2], v3[1]/v3[2]]) for v3 in v3List]
    return polygon2d(v2List, color = color)

plotList(xList, 'red')

In [ ]:
# See the 45 degree rotration
b0 = A1 * x0
b1 = A1 * x1
b2 = A1 * x2
b3 = A1 * x3
bList = [b0, b1, b2, b3]
show(bList)
# Better way of creating bList: Using List Comprehension
bList = [A1 * x for x in xList]
show(bList)
plotList(bList, color='green') 

### Rotation + Translation
Adding $2$ and $1$ to the $x$ and $y$ coordinates. 

In [ ]:
# Add a translation to the rotation
A2 = matrix(SR, 3, 3, [[1/sqrt(2), -1/sqrt(2), 2], [1/sqrt(2), 1/sqrt(2), 1], [0, 0, 1]])
show(A2)

In [ ]:
# Rotated and translated square
bList = [A2 * x for x in xList]
show(bList)
plotList(bList, color='green')

### Rotation, Translation & Projection
Add a projective operation as well. Now the property of homegeneous coordinates will come into play: Scaled coordinates represent the same point. We scale the $\boldsymbol{b}$ vectors by its last coordinate to make it unity.

In [ ]:
# Add a projection to it
A3 = matrix(SR, 3, 3, [[1/sqrt(2), -1/sqrt(2), 2], [1/sqrt(2), 1/sqrt(2), 1], [1/2, 3/2, 1]])
show(A3)

In [ ]:
bList = [A3 * x for x in xList]
show(bList)
plotList(bList, color='green')

## Solving for the Matrix
We have four equations:  

$$ \boldsymbol{A_3 x_i =  b_i} \quad i=0,1,2,3$$

Each one is really three equations. 

When we actually do perspective correction, we know $\boldsymbol {b_i}$ and $\boldsymbol {x_i}$. But we don't know the elements of $\boldsymbol{A_3}$. 

$$ \boldsymbol {A_3} = \left(\begin{array}{rrr}
a_{00} & a_{01} & b_{02} \\
a_{10} & a_{11} & b_{12} \\
c_{20} & c_{21} & 1
\end{array}\right) $$

Let's write down the eight equations, and solve them, the "kindergarten" way.  
*** Why only eight equations? Why not nine? ***

In [ ]:
# Try to solve for A: The "kindergarten" way!
eqList = [[row == bList[j][i] for i, row in enumerate(A*x)] for j, x in enumerate(xList)]
eqns = flatten(eqList)
shown = [show(eq) for eq in eqns]

In [ ]:
# Solve the equations
soln = solve(eqns, vars)
show(A)
shown = [[show(s) for s in sol] for sol in soln]
print("The original matrix A3 is:")
show(A3)

In [ ]:
# Solve again, after scaling by the third element (because b's use homogenous coordinates)
# Scaling of a coordinate does not change its location
eqList = [[row == bList[j][i]/bList[j][2] for i, row in enumerate(A*x)] for j, x in enumerate(xList)]
eqns = flatten(eqList)
shown = [show(eq) for eq in eqns]
soln1 = solve(eqns, vars)
show(A)
shown = [[show(s) for s in sol] for sol in soln1]
show(A3)
print("Is the scaled solution the same as unscaled one?", soln == soln1)

In [ ]:
# Another function to extract the matrix form from a system of linear equations
def getMatrixForm(eqList, vars):
    A = matrix([[eqn.lhs().coefficient(v) for v in vars] for eqn in eqList])
    b = matrix([[eqn.rhs()] for eqn in eqList])
    return (A, b)

In [ ]:
show(getMatrixForm(eqns, vars))

## Solving for the Matrix: Our Way
Since we have four equations:  

$$ \boldsymbol{A_3  x_i =  b_i} \quad i=0,1,2,3$$

we can combine them into one matrix equation. Place the four vectors $\boldsymbol {x_i}$ in a matrix $\boldsymbol X$ and the four vectors $\boldsymbol {b_i}$ in another matrix $\boldsymbol B$. They are both $3\times 4$ matrices now.

$$\boldsymbol{AX = B} $$

If we could find an "inverse" for $X$, we are done. But $X$ is not square, but has 3 rows and 4 columns $\implies$ wide, full-rank matrix. Therefore it has a right-inverse.

Knowing that $XX^T$ is a square, full-rank matrix. So we can write:

$$\boldsymbol{\left(XX^T\right)\left(XX^T\right)^{-1} = I}$$

Regrouping:
$$\boldsymbol{X\left(X^T\left(XX^T\right)^{-1}\right) = I}$$

Therefore, we can treat $\boldsymbol{X^T\left(XX^T\right)^{-1}}$ as $\boldsymbol{X^{-1}_\text{Right}}$.

$$\boldsymbol{ A_4 = B\, X^T\left(XX^T\right)^{-1}}$$

$\boldsymbol{A_4}$ is the solution.

In [ ]:
# Solve for A in the matrix way
X = matrix(xList).transpose()
B = matrix(bList).transpose()
show("A3 = ", A3, " X =", X, " B = ", B)
print ("Is A3 * X == B? ", A3 * X == B)

In [ ]:
# Get the right inverse of X
A4 = B * X.transpose() * ( X * X.transpose() ).inverse()
show(A4)
print ("Is A4 (matrix way) the same as A3 (kindergarten)?", A4 == A3)

## Discussion
Remember, $\boldsymbol b_i$s are the points on the image and $\boldsymbol x_i$ are the points after taking out the distortion due to the camera prespective. Since we wrote $ \boldsymbol{A}\, \boldsymbol x_i = \boldsymbol b_i \quad i=0,1,2,3$, what we really need is $ \boldsymbol{A}^{-1}$.

What if we wrote the equation differently?

$$ \boldsymbol{C}\, \boldsymbol b_i = \boldsymbol x_i \quad i=0,1,2,3$$

In other words, $\boldsymbol{C}$ is the matrix that will take us from a distorted image to the perspective-rectified image. Clearly, $\boldsymbol{C} = \boldsymbol{A}^{-1}$.

In the matrix way of solving it, we can write:

$$ \boldsymbol{CB = X} $$

$$ \boldsymbol{C = X\,B^{-1} = X\,B^T\,(BB^T)^{-1}}$$

Let's verify if $C=A_4^{-1}$.

In [ ]:
C = X * B.transpose() * ( B * B.transpose() ).inverse()
A4Inv = A4.inverse()
C = C.n(digits=4)
show("C = ", C)
A4Inv = A4Inv.n(digits=4)
show("$A4 inverse = ", A4Inv)

print ("Is C == A4Inv? ", C == A4Inv)
print ("Relative size of the difference: [C-A4Inv).norm() / C.norm()] = ", \
( (C-A4Inv).norm()/C.norm()*100 ).n(digits=2), "%")

In [ ]:
# Redo it by ensuring that the last diagonal element is 1
C = X * B.transpose() * ( B * B.transpose() ).inverse()
C = C / C[2,2]
C = C.n(digits=4)
show("C = ", C)
show("$A4 inverse = ", A4Inv)
print ("Relative size of the difference: [C-A4Inv).norm() / C.norm()] = ", \
( (C-A4Inv).norm()/C.norm()*100 ).n(digits=2), "%")

## Last Words
What do we do once we have $ \boldsymbol{A}^{-1}$ or $ \boldsymbol{C}$?  

We take the photo to be corrected, identify the location of each pixel as a vector $(x, y, 1)$, transform it to $(x', y', 1)$, and assign the same image value to it. In other words, the RGB value that was at $(x, y)$ of the original image will get assigned to the pixel at $(x', y')$ in the new image. Obviously, some distortions may happen, and we may have to apply advanced techniques like anti-aliasing or (spline) smoothing.

In [ ]:
# x1List contains the x' (perspective-corrected) vectors
x1List = [C * b for b in bList]
plotList(x1List, color='blue')